# Accessing and managing users
Users are an indispensible part of your web GIS. As the number of users grow, you can see value in automating your management tasks such as provisioning licenses, privileges, creating and removing user accounts etc. The `gis` module provides you with `User` and `UserManager` classes to respresent users as objects and help you accomplish the most common tasks. 

As you might have seen the pattern with `ContentManager` and `Item` objects, the `UserManager` object is a resource manager that gives you access to `User` objects. You access a `UserManager` object not by instantiating that class through its constructor, but by accessing the `users` property of your `GIS` object. This is the typical pattern of usage throughout the `gis` module.

<a id="about-your-account"></a>
## About your account
Let us get to know a bit about our logged in account before we observe how to manage other user accounts. Let us import the `GIS` class from `gis` module and connect to an ArcGIS Enterprise:

In [ ]:
from arcgis.gis import GIS
gis = GIS(profile="your_online_profile")

You can access your user account by accessing `me` property as shown below:

In [ ]:
me = gis.users.me
me

Similar to `Item` objects, when using the Jupyter notebook IDE, you can visualize `User` objects in rich HTML representation with thumbnails and attribute information.

<a id="properties-of-a-user-object"></a>
## Properties of a `User` object
You can query much more information about the user account as properties on the `User` object:

In [ ]:
me.access

You can find out when an account was last active and determine if an account was abandoned and remove it if necessary.

In [ ]:
import time
# convert Unix epoch time to local time
created_time = time.localtime(me.created/1000)
print("Created: {}/{}/{}".format(created_time[0], created_time[1], created_time[2]))

last_accessed = time.localtime(me.lastLogin/1000)
print("Last active: {}/{}/{}".format(last_accessed[0], last_accessed[1], last_accessed[2]))

Let us print some more information about this account

In [ ]:
print(me.description, " ", me.email, " ", me.firstName, " ", me.lastName, " ", me.fullName)
print(me.level, " ", me.mfaEnabled, " ", me.provider, " ", me.userType)

You can determine the groups the user is a member of:

In [ ]:
user_groups = me.groups
print("Member of " + str(len(user_groups)) + " groups")

# groups are returned as a dictionary. Lets print the first dict as a sample
user_groups[0]

<a id="searching-for-user-accounts"></a>
## Searching for user accounts
The `search()` method of `UserManager` class helps you search for users of the org. The `query` parameter in the `search()` method accepts standard [ArcGIS REST API queries](https://developers.arcgis.com/rest/users-groups-and-items/search-reference.htm) and behaves similar to the search method on `ContentManager` and `GroupManager` classes. To illustrate this better, let us search ArcGIS Online as there are many more users available there.

In [ ]:
# search the users whose email address ends with esri.com
esri_accounts = gis.users.search(query='cedric',outside_org=True)
len(esri_accounts)
esri_accounts

Each element in the list returned is a `User` object that you can query.

In [ ]:
# lets filter out Esri curator accounts from this list
cede_account = [acc for acc in esri_accounts if "despi" in acc.username]
cede_account

In [ ]:
cede_account[0]

Once you know a user's username, you can access that object using the **`get()`** method. Let us access the Esri curator account for historical maps

In [ ]:
cede_account = ago_gis.users.get(username='cede_account')
cede_account

> Note that we specified `arcgis` as the `provider` argument. If you were creating accounts from your enterprise credential store, you would specify this value as `enterprise` and use the `idpUsername` parameter to specify the username of the user in that credential store. To learn more about this configuration, refer to this help topic on [setting up enterprise logins](https://enterprise.arcgis.com/en/portal/latest/administer/windows/about-configuring-portal-authentication.htm#ESRI_SECTION1_83F7B85FEF594A6B96997AF3CADF3D38).

Note, the `role` parameter was specified as `org_user`. This takes us to the next section on `Role` and `RoleManager` objects.

<a id="about-user-roles"></a>
### About user roles
ArcGIS provides a security concept called roles which defines the privileges a user has within an organization. By default, your org has 3 roles - `org_user`, `org_publisher` and `org_admin`. You can refer to [this topic on organizational roles](https://doc.arcgis.com/en/arcgis-online/reference/roles.htm) to learn about these three roles and their privileges. In summary, a user role can be an active user of the org, create items, join groups and share content. A publisher role has all of user privileges and can create hosted content and perform analysis. An administrator role has all possible privileges. 

Depending on the size of your org and the security needs, you can customize this and create any number of roles with fine grained privileges. For reference on custom roles in an org, refer to [this doc](https://doc.arcgis.com/en/arcgis-online/reference/roles.htm#ESRI_SECTION1_7071F89DE04B448CA833A4164A98DF94)

To know about the role of a `User` object, you can query the `role` property:

In [ ]:
me.role


<a id ="listing-all-the-custom-roles-in-an-org"></a>
#### Listing all the custom roles in an org
When migrating users from one org to another or even to duplicate an org on new infrastructure, you would go through the process of cloning the users and their roles. For this, you can get the list of roles using the `all()` method on the `RolesManager` resource object:

In [ ]:
gis.users.roles.all(max_roles=50)

<a id = "deleting-user-accounts"></a>
## Deleting user accounts
You can delete user accounts by calling the `delete()` method on a `User` object from an account that has administrator privileges. However, deleting raises important questions such as what happens to the content owned by that user? Further, ArcGIS does not allow you to delete users until you have dealt with that users' items and groups. Thus as an administrator, it becomes useful to list and view the content owned by any user in your org.

<a id = "accessing-user-content"></a>
### Accessing user content
Once you have a `User` object, you can view the folders and items owned by the user by querying the `folders` property and calling the `items()` method.

In [ ]:
userslist =  gis.users.search(query ="*", sort_field = "created", sort_order="asc", max_users=5)
for user in userslist:
    display(user)

In [ ]:
# list all items belonging to a user
item_list_rootfolder = userslist[0].items()
print("Total number of items in root folder: " + str(len(item_list_rootfolder)))
item_list_rootfolder[0]

Thus using a `GIS` object created with an account that has admin privileges, you were able to query the contents of another user without knowing that user's password or logging in as that user.

<a id="reassigning-user-content"></a>
### Reassigning user content
As an administrator, you have the privileges to list and view other users' content. When the time comes to delete a user account, you can filter these items and choose to preserve some of them and delete the rest.

In [ ]:
for item in item_list_rootfolder:
        print(item..reassign_to(target_owner=gis.users.me.username))

You can reassign specific items to another user by calling the `reassign_to()` method on that `Item` object. 
It's also possible to just add the reassignement during the item deletion

In [ ]:
xxxx_user.delete(reassign_to='cede_esrich')